# 4.C. Recommender -- Album Level

We'll test out our album review recommendation engine in this notebook before incorporating into a hybrid approach in notebook **4D**. The code is replicated in the following notebook.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import lyricsgenius
import re
import sys
import spotipy
import spotipy.util as util
from sklearn.compose import ColumnTransformer, make_column_transformer
from pyjarowinkler import distance
import nltk

import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

import nltk
from nltk.stem import PorterStemmer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB


In [2]:
albums = pd.read_csv('./combining/albumdata/review_data_for_recommender.csv')

In [3]:
albums = albums[albums.combined_reviews_clean.notnull()]

In [4]:
albums = albums.reset_index()

In [5]:
tfidf = TfidfVectorizer(
    min_df = 2,
    max_features = 1000,
    ngram_range=(1, 2),
)

tfidf_matrix = tfidf.fit_transform(albums.combined_reviews_clean)

In [6]:
from sklearn.metrics.pairwise import linear_kernel 

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 
results = {}
for idx, row in albums.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-10:-1] 
    similar_items = [(cosine_similarities[idx][i], albums['artist_album_clean_key'][i]) for i in similar_indices] 
    results[row['artist_album_clean_key']] = similar_items[1:]

In [7]:
def item(id):  
    return albums.loc[albums['artist_album_clean_key'] == id]['artist_album_clean_key']
# Just reads the results out of the dictionary.def 
def recommend(item_id, num):   
    recs = results[item_id][:num]   
    for rec in recs: 
        print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [10]:
albums[albums['artist'].str.contains('chance')]

,index,artist_album_clean_key,artist,album,combined_reviews_clean,pitch_date,pitch_score,needle_score
715,727,chance the rapper|acid rap,chance the rapper,acid rap,dont fooled title chance latest self released ...,2013,8.400000,6.000000
716,728,chance the rapper|coloring book,chance the rapper,coloring book,chance stranger attention mixtape stepping sto...,2016,9.100000,0.000000
717,729,chance the rapper|merry christmas lil mama,chance the rapper,merry christmas lil mama,chance performed moments everything snug safe ...,2016,8.100000,5.394723
718,730,chance the rapper|the big day,chance the rapper,the big day,chance sprawling minute exuberant often wonder...,2019,6.900000,0.000000
5360,5410,chance the rapper|10 day,chance the rapper,10 day,patrick first bust open chance rr chicago emce...,NaN,6.952916,5.394723


In [18]:
recommend('chance the rapper|acid rap', 10)

718    Recommended: chance the rapper|the big day (sc...
Name: artist_album_clean_key, dtype: object
716    Recommended: chance the rapper|coloring book (...
Name: artist_album_clean_key, dtype: object
1179    Recommended: donnie trumpet and the social exp...
Name: artist_album_clean_key, dtype: object
6901    Recommended: jeru the damaja|divine design (sc...
Name: artist_album_clean_key, dtype: object
760    Recommended: childish gambino|because the inte...
Name: artist_album_clean_key, dtype: object
2507    Recommended: macklemore and ryan lewis|this un...
Name: artist_album_clean_key, dtype: object
2682    Recommended: mick jenkins|the water (score:0.4...
Name: artist_album_clean_key, dtype: object
541    Recommended: bobby womack|the bravest man in t...
Name: artist_album_clean_key, dtype: object
